# Face Recognation 
Project based on: [Build a Deep Facial Recognition App](https://www.youtube.com/watch?v=bK_k7eebGgc&list=PLgNJO2hghbmhHuhURAGbe6KWpiYZt0AMH)

In [1]:
pip install labelme tensorflow opencv-python matplotlib albumentations

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


# 1.2 Collect images using openCV

In [1]:
import os
import time
import uuid # for unique id
import cv2

In [2]:
IMAGES_PATH = os.path.join('data', 'images')
NUM_IMAGES_TO_COLLECT = 20

In [3]:
# Conection to video camera and get images
cap = cv2.VideoCapture(0) # if you are on winodws try 1 instead of 0
for imgnum in range(NUM_IMAGES_TO_COLLECT):
    print('Collecting image {}'.format(imgnum))
    ret, frame = cap.read()
    imgname = os.path.join(IMAGES_PATH,f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(imgname, frame)
    cv2.imshow('frame', frame)
    time.sleep(0.5)

    if cv2.waitKey(1) & 0xFF == ord('q'): # turn off the camera
        break
cap.release()
cv2.destroyAllWindows()

# Annotate images with labelMe

In [2]:
/Users/asialalala/.local/pipx/venvs/labelme/bin/labelme

NameError: name 'Users' is not defined